In [1]:
import torch.nn as nn

# Residual block
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x):
        return self.fn(x) + x

# ConvMixer model with hard-coded parameters
def ConvMixer():
    dim = 256          # Embedding dimension
    depth = 8          # Number of ConvMixer blocks
    kernel_size = 5    # Kernel size for depthwise convolution
    patch_size = 4     # Patch size for initial convolution
    n_classes = 10     # CIFAR-10 has 10 classes

    return nn.Sequential(
        nn.Conv2d(3, dim, kernel_size=patch_size, stride=patch_size),
        nn.GELU(),
        nn.BatchNorm2d(dim),
        *[nn.Sequential(
                Residual(nn.Sequential(
                    nn.Conv2d(dim, dim, kernel_size, groups=dim, padding="same"),
                    nn.GELU(),
                    nn.BatchNorm2d(dim)
                )),
                nn.Conv2d(dim, dim, kernel_size=1),
                nn.GELU(),
                nn.BatchNorm2d(dim)
        ) for _ in range(depth)],
        nn.AdaptiveAvgPool2d((1, 1)),
        nn.Flatten(),
        nn.Linear(dim, n_classes)
    )

In [2]:
# Load the model
import torch

# Define the path to the model
device = "cuda" 

# Load the model
model = torch.load('/home/j597s263/scratch/j597s263/Models/ConvModels/Conv_Imagenette.mod', weights_only=False, map_location="cuda")
model = model.to(device)
model.eval()  

print("Model loaded successfully!")

Model loaded successfully!


In [3]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt
import random

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor()
])

dataset = datasets.Imagenette(root='/home/j597s263/scratch/j597s263/Datasets/imagenette', download=False, transform=transform)

random.seed(42) 
indices = list(range(len(dataset)))
random.shuffle(indices)

# Split shuffled indices into training and testing
train_indices = indices[:7568]
test_indices = indices[7568:8522]
attack_indices = indices[8522:]

# Create Subsets
train_data = Subset(dataset, train_indices)
test_data = Subset(dataset, test_indices)
attack_data = Subset(dataset, attack_indices)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  # Shuffle within batches
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)  # No shuffle for test set
attack_loader = DataLoader(attack_data, batch_size=1, shuffle=False)  # Batch size 1 for mask intersection

In [4]:
import torch
import numpy as np
from captum.attr import IntegratedGradients
import os

# Define the Integrated Gradients method
ig = IntegratedGradients(model)

# Function to compute and save explanations as .npy
def compute_and_save_explanations_with_labels(data_loader, save_dir):
    """
    Compute Integrated Gradient explanations for all images in the dataset and save as .npy files.

    Args:
        data_loader (DataLoader): DataLoader for the dataset to explain.
        save_dir (str): Directory to save the explanations as .npy files.
    """
    os.makedirs(save_dir, exist_ok=True)

    for batch_idx, (images, labels) in enumerate(data_loader):
        images, labels = images.to(device), labels.to(device)

        # Compute attributions
        attributions, delta = ig.attribute(
            inputs=images,
            target=labels,  # Use ground-truth labels for explanations
            baselines=torch.zeros_like(images).to(device),  # Baseline: black image
            return_convergence_delta=True,
        )

        # Save explanations for each image in the batch
        for i in range(images.size(0)):
            # Combine label and explanation
            label = labels[i].item()
            explanation = attributions[i].cpu().numpy()  # Shape: (3, 224, 224)

            # Create a label array of shape (1, 224, 224)
            label_array = np.full((1, explanation.shape[1], explanation.shape[2]), label, dtype=np.float32)

            # Concatenate label and explanation to shape (4, 224, 224)
            explanation_with_label = np.concatenate([label_array, explanation], axis=0)

            # Save as .npy
            save_path = os.path.join(save_dir, f"explanation_{batch_idx * images.size(0) + i}.npy")
            np.save(save_path, explanation_with_label)

        print(f"Processed batch {batch_idx + 1}/{len(data_loader)}")

    print(f"Explanations saved to {save_dir}")


# Define the directory to save explanations
save_dir = "/home/j597s263/scratch/j597s263/Datasets/Explanation_values/IG_ConvImg.npy"

# Compute and save explanations
compute_and_save_explanations_with_labels(attack_loader, save_dir)

Processed batch 1/947
Processed batch 2/947
Processed batch 3/947
Processed batch 4/947
Processed batch 5/947
Processed batch 6/947
Processed batch 7/947
Processed batch 8/947
Processed batch 9/947
Processed batch 10/947
Processed batch 11/947
Processed batch 12/947
Processed batch 13/947
Processed batch 14/947
Processed batch 15/947
Processed batch 16/947
Processed batch 17/947
Processed batch 18/947
Processed batch 19/947
Processed batch 20/947
Processed batch 21/947
Processed batch 22/947
Processed batch 23/947
Processed batch 24/947
Processed batch 25/947
Processed batch 26/947
Processed batch 27/947
Processed batch 28/947
Processed batch 29/947
Processed batch 30/947
Processed batch 31/947
Processed batch 32/947
Processed batch 33/947
Processed batch 34/947
Processed batch 35/947
Processed batch 36/947
Processed batch 37/947
Processed batch 38/947
Processed batch 39/947
Processed batch 40/947
Processed batch 41/947
Processed batch 42/947
Processed batch 43/947
Processed batch 44/9